<a href="https://colab.research.google.com/github/sseunghyuns/deeplearning/blob/master/horses_or_humans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision with CNNs

## 라이브러리 가져오기

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

## 데이터셋 가져오기

In [2]:
dataset_name = 'horses_or_humans'

In [3]:
train_dataset,info = tfds.load(name=dataset_name, split = 'train[:80%]',with_info = True)
valid_dataset = tfds.load(name=dataset_name, split = 'train[-20%:]')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteS7ODAX/horses_or_humans-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteS7ODAX/horses_or_humans-test.tfrecord


Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


* 텐서플로 데이터셋의 80%는 train data, 20%는 test data로 사용하였습니다.

## 데이터 전처리

In [4]:
def preprocess(features):
    # YOUR CODE HERE
    img , lbl = tf.cast(features['image'],tf.float32) / 255.0 , features['label']
    img = tf.image.resize(img, (300,300))
    return img, lbl

* 데이터를 image, label로 나누는 함수를 정의하였습니다.
* image 데이터를 0~1사이 값으로 정규화하였습니다.
* image 데이터 크기를 (300,300)으로 통일하였습니다.

In [5]:
batch_size = 32
total_image_size = info.splits['train'].num_examples
steps_per_epoch = (total_image_size * 0.8) // batch_size + 1
validation_steps = (total_image_size * 0.2) // batch_size + 1

In [6]:
train_data = train_dataset.map(preprocess).batch(batch_size).repeat() # 데이터 전처리 함수를 적용하고 배치 사이즈만큼 학습을 할 수 있도록 하였습니다.
valid_data = valid_dataset.map(preprocess).batch(batch_size).repeat()

## CNN 모델 생성

In [7]:
model = Sequential([
                    Conv2D(128, (3,3), input_shape = (300,300,3), activation = 'relu'),
                    MaxPooling2D(),                        

                    Conv2D(64, (3,3), activation = 'relu'),
                    MaxPooling2D(),

                    Conv2D(32, (3,3), activation = 'relu'),
                    MaxPooling2D(),

                    Conv2D(16, (3,3), activation = 'relu'),
                    MaxPooling2D(),

                    Flatten(),
                    Dropout(0.5), # 과적합 방지
                    Dense(512, activation = 'relu'),
                    Dense(258, activation = 'relu'),
                    Dense(2, activation = 'softmax')
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 16)        4

In [8]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

check_path = 'ck.ckpt'
check_point = ModelCheckpoint(
    filepath = check_path,
    save_best_only = True,
    save_weights_only = True,
    monitor = 'val_loss',
    verbose = 1
)

In [9]:
model.fit(train_data,
          validation_data = (valid_data),
          epochs = 10,
          steps_per_epoch = steps_per_epoch,
          validation_steps = validation_steps,
          callbacks = [check_point])

Epoch 1/10
 2/26 [=>............................] - ETA: 4s - loss: 0.6926 - acc: 0.4844WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1308s vs `on_train_batch_end` time: 0.2087s). Check your callbacks.


26/26 [==============================] - ETA: 0s - loss: 0.5483 - acc: 0.6740
Epoch 00001: val_loss improved from inf to 0.19495, saving model to ck.ckpt
26/26 [==============================] - 12s 454ms/step - loss: 0.5483 - acc: 0.6740 - val_loss: 0.1949 - val_acc: 0.8976
Epoch 2/10
26/26 [==============================] - ETA: 0s - loss: 0.1487 - acc: 0.9440
Epoch 00002: val_loss did not improve from 0.19495
26/26 [==============================] - 10s 390ms/step - loss: 0.1487 - acc: 0.9440 - val_loss: 0.3996 - val_acc: 0.8537
Epoch 3/10
26/26 [==============================] - ETA: 0s - loss: 0.1392 - acc: 0.9501
Epoch 00003: val_loss improved from 0.19495 to 0.05050, saving model to ck.ckpt
26/26 [==============================] - 10s 387ms/step - loss: 0.1392 - acc: 0.9501 - val_loss: 0.0505 - val_acc: 0.9756
Epoch 4/10
26/26 [==============================] - ETA: 0s - loss: 0.1034 - acc: 0.9684
Epoch 00004: val_loss did not improve from 0.05050
26/26 [========================

In [10]:
model.load_weights(check_path) # 성능이 가장 좋았던 weigth를 로드합니다. 